# Finding Common Attributes in GroupBy
Thanks to my colleague Neal for showing me this technique.

In [1]:
import pandas as pd

Let's say we have a bunch of logs containing usernames and activity IP addresses.

We want to find which IP addresses are shared by N or more users.

In [2]:
data = [
    ['alice','10.1.1.1'],
    ['alice','10.1.1.1'],
    ['bob','192.168.1.2'],
    ['cate','172.16.1.2'],
    ['cate','10.1.1.1'],
    ['diane','172.16.100.2'],
    ['diane','172.16.1.2'],
    ['edward','192.168.200.200'],
    ['edward','172.16.100.2'],
    ['frank','10.1.1.1']
]

In [3]:
df = pd.DataFrame(data,columns=['user','ip'])

In [4]:
df

,user,ip
0,alice,10.1.1.1
1,alice,10.1.1.1
2,bob,192.168.1.2
3,cate,172.16.1.2
4,cate,10.1.1.1
5,diane,172.16.100.2
6,diane,172.16.1.2
7,edward,192.168.200.200
8,edward,172.16.100.2
9,frank,10.1.1.1


Here's a fairly standard "aggregate and count" method, but neither version tells us what addresses are shared by N or more users, without more work:

In [5]:
df.groupby('ip').size()

ip
10.1.1.1           4
172.16.1.2         2
172.16.100.2       2
192.168.1.2        1
192.168.200.200    1
dtype: int64

In [6]:
df.groupby(['user','ip']).size()

user    ip             
alice   10.1.1.1           2
bob     192.168.1.2        1
cate    10.1.1.1           1
        172.16.1.2         1
diane   172.16.1.2         1
        172.16.100.2       1
edward  172.16.100.2       1
        192.168.200.200    1
frank   10.1.1.1           1
dtype: int64

Here's what's actually inside a GroupBy object. It groups the DataFrame into rows based on indexes referencing the original DataFrame:

In [7]:
df.groupby('ip').groups

{'10.1.1.1': Int64Index([0, 1, 4, 9], dtype='int64'),
 '172.16.1.2': Int64Index([3, 6], dtype='int64'),
 '172.16.100.2': Int64Index([5, 8], dtype='int64'),
 '192.168.1.2': Int64Index([2], dtype='int64'),
 '192.168.200.200': Int64Index([7], dtype='int64')}

Let's say we want to see any IPs with at least two common users:

In [8]:
minimum_users_sharing_ip = 2

Putting our code inside parentheses is a convenience that allows us to read chained functions more easily and use comments to better understand it:

In [15]:
(
df.groupby('ip')
  .filter(lambda x: len(set(x['user'])) >= minimum_users_sharing_ip)
  .reset_index()
  .groupby('ip')
  .agg({'user':'unique'})
)

,user
ip,
10.1.1.1,"[alice, cate, frank]"
172.16.1.2,"[cate, diane]"
172.16.100.2,"[diane, edward]"


So, why does that work? The `filter` function iterates the `GroupBy` object groups (shown above) and looks at the number of unique users in each row after grouping by IP. If the number is greater than or equal to 2, it includes those rows in the filtered DataFrame:

In [16]:
(
df.groupby('ip')
  .filter(lambda x: len(set(x['user'])) >= minimum_users_sharing_ip)
  #.reset_index().groupby('ip').agg({'user':'unique'})
)

,user,ip
0,alice,10.1.1.1
1,alice,10.1.1.1
3,cate,172.16.1.2
4,cate,10.1.1.1
5,diane,172.16.100.2
6,diane,172.16.1.2
8,edward,172.16.100.2
9,frank,10.1.1.1


The subsequent `.reset_index().groupby('ip')` part groups the new DataFrame again by IP:

In [17]:
(
df.groupby('ip')
  .filter(lambda x: len(set(x['user'])) >= minimum_users_sharing_ip)
  .reset_index().groupby('ip')
  .groups
  #.agg({'user':'unique'})
)

{'10.1.1.1': Int64Index([0, 1, 3, 7], dtype='int64'),
 '172.16.1.2': Int64Index([2, 5], dtype='int64'),
 '172.16.100.2': Int64Index([4, 6], dtype='int64')}

And finally, the `.agg({'user':'unique'})` function aggregates the object using a built-in function that groups unique users together.

We can turn that into a function:

In [18]:
def find_minimum_intersecting_users(df,minimum_users_sharing_ip):
    return \
        (
        df.groupby('ip')
          .filter(lambda x: len(set(x['user'])) >= minimum_users_sharing_ip)
          .reset_index()
          .groupby('ip')
          .agg({'user':'unique'})
        )

In [20]:
find_minimum_intersecting_users(df,2)

,user
ip,
10.1.1.1,"[alice, cate, frank]"
172.16.1.2,"[cate, diane]"
172.16.100.2,"[diane, edward]"


In [21]:
find_minimum_intersecting_users(df,3)

,user
ip,
10.1.1.1,"[alice, cate, frank]"
